In [1]:
import mysql.connector
import pandas as pd
from config import *
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error

ModuleNotFoundError: No module named 'mysql'

In [2]:
my_user='tec_A00827376'
secret='baBU0A1r!X3L2UU0'
config = {
    'user': my_user,
    'password': secret,
    'host': 'topline.dwh.suite.ghp.mx',
    'database': 'dwh',
    'raise_on_warnings': True
}

cnx = mysql.connector.connect(**config)

In [3]:
DM_Chains = pd.read_sql("SELECT * FROM  DM_Chains", cnx)

DM_Venues = pd.read_sql("SELECT * FROM  DM_Venues", cnx)

DM_Brands = pd.read_sql("SELECT * FROM  DM_Brands", cnx)

DM_Verticals = pd.read_sql("SELECT * FROM  DM_Verticals", cnx)

DM_Cities = pd.read_sql("SELECT * FROM  DM_Cities", cnx)

DM_States = pd.read_sql("SELECT * FROM  DM_States", cnx)

DM_Countries = pd.read_sql("SELECT * FROM  DM_Countries", cnx)

DM_Business_Dates = pd.read_sql("SELECT * FROM  DM_Business_Dates", cnx)

factTable = pd.read_sql("SELECT * FROM  FT_Nights", cnx)

e:\Programas\Anaconda\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
e:\Programas\Anaconda\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
e:\Programas\Anaconda\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
e:\Programas\Anaconda\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 c

In [4]:
DM_Brands.rename(columns = {'name':'brand_name'}, inplace= True)

DM_Chains.rename(columns = {'name':'chain_name'}, inplace= True)

DM_Cities.rename(columns = {'name':'city_name'}, inplace= True)

DM_Countries.rename(columns = {'name':'country_name','iso_code':'country_iso_code'}, inplace= True)

DM_States.rename(columns = {'name':'state_name','iso_code':'state_iso_code'}, inplace= True)

DM_Venues.rename(columns = {'name':'venue_name'}, inplace= True)

DM_Verticals.rename(columns = {'name':'vertical_name'}, inplace= True)


In [5]:
df = factTable.merge(DM_Venues, on='venue_id')

df = df.merge(DM_Brands, on="brand_id")

df = df.merge(DM_Chains, on="chain_id")

df = df.merge(DM_Verticals, on="vertical_id")

df = df.merge(DM_Cities, on="city_id")

df = df.merge(DM_States, on="state_id")

df = df.merge(DM_Countries, on="country_id")

df = df.merge(DM_Business_Dates, on="business_date_id")

In [6]:
df = df[df.found != 0]
df = df[df.active != 0]

In [7]:
datos = df.drop(['week_day_name', 'month_name_short','month_name','iso_week_year','available_rooms','country_id','country_iso_code','day', 'month', 'year', 'week_day',
 'day_of_year','venue_id', 'business_date_id', 'found','currency_id','brand_id','vertical_id', 'region_id', 'city_id','chain_id', 'state_id','state_iso_code', 
 'iso_week_number','quarter', 'semester', 'incode','latitude','longitude'],axis= 1)

In [8]:
datos = datos[datos["venue_name"].str.contains("Wyndham") == True]

In [9]:
datos.venue_name.unique()

array(['Wyndham Garden Playa del Carmen',
       'Wyndham Garden Guadalajara Andares',
       'Wyndham Garden Monterrey Valle Real',
       'Microtel Inn & Suites by Wyndham Chihuahua',
       'Microtel Inn & Suites by Wyndham Ciudad Juárez',
       'Microtel Inn & Suites by Wyndham Toluca'], dtype=object)

## Exploración de datos

In [10]:
datos.describe()

,night_id,rooms_available,rooms_occupied,rooms_revenue,compset_rooms_available,compset_rooms_occupied,compset_rooms_revenue,active
count,5205.000000,5205.000000,5205.000000,5205.000000,5205.000000,5205.000000,5.205000e+03,5205.0
mean,23030.120461,132.760231,50.185975,52290.286013,656.075889,254.542555,2.677102e+05,1.0
std,12276.225359,44.673215,37.014248,47627.971030,177.814075,158.810078,2.108280e+05,0.0
min,8821.000000,0.000000,0.000000,-9028.640000,0.000000,0.000000,-3.266100e+03,1.0
25%,12773.000000,108.000000,22.000000,18179.760000,586.000000,145.000000,1.239207e+05,1.0
50%,17602.000000,113.000000,45.000000,37702.380000,595.000000,236.000000,2.164067e+05,1.0
75%,31251.000000,186.000000,74.000000,75135.120000,766.000000,334.000000,3.511061e+05,1.0
max,46746.000000,196.000000,177.000000,399920.670000,1009.000000,951.000000,1.623219e+06,1.0


## Creación de modelo

In [11]:
datos.corr()

,night_id,rooms_available,rooms_occupied,rooms_revenue,compset_rooms_available,compset_rooms_occupied,compset_rooms_revenue,active
night_id,1.000000,-0.395338,-0.288664,-0.319548,0.106411,-0.073157,-0.135673,NaN
rooms_available,-0.395338,1.000000,0.446854,0.433716,0.119442,0.193882,0.191279,NaN
rooms_occupied,-0.288664,0.446854,1.000000,0.934125,0.047097,0.763274,0.723661,NaN
rooms_revenue,-0.319548,0.433716,0.934125,1.000000,0.200215,0.800935,0.832417,NaN
compset_rooms_available,0.106411,0.119442,0.047097,0.200215,1.000000,0.396244,0.412761,NaN
compset_rooms_occupied,-0.073157,0.193882,0.763274,0.800935,0.396244,1.000000,0.961643,NaN
compset_rooms_revenue,-0.135673,0.191279,0.723661,0.832417,0.412761,0.961643,1.000000,NaN
active,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
#Declaramos nuestras variables independientes 
X = pd.DataFrame(datos[['rooms_revenue', 'compset_rooms_available', 'compset_rooms_revenue']])
X = sm.add_constant(X)
#Definimos la variable dependiente
Y = pd.DataFrame(datos['rooms_occupied']) 

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=42)

## Modelo lineal1

In [14]:
model1 = sm.OLS(y_train,X_train).fit()

In [15]:
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:         rooms_occupied   R-squared:                       0.896
Model:                            OLS   Adj. R-squared:                  0.896
Method:                 Least Squares   F-statistic:                 1.044e+04
Date:                Tue, 25 Oct 2022   Prob (F-statistic):               0.00
Time:                        11:30:49   Log-Likelihood:                -14157.
No. Observations:                3643   AIC:                         2.832e+04
Df Residuals:                    3639   BIC:                         2.835e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

In [16]:
mean_squared_error(y_test,model1.predict(X_test))


158.48179157285247

## Modelo Lineal2

In [17]:
#Declaramos nuestras variables independientes 
X = pd.DataFrame(datos[['rooms_revenue', 'rooms_available']])
X = sm.add_constant(X)
#Definimos la variable dependiente
Y = pd.DataFrame(datos['rooms_occupied']) 

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=42)

In [19]:
model2=LinearRegression()
model2.fit(X_train,y_train)
print(model2.intercept_, model2.coef_, model2.score(X_train,y_train))

[7.1489941] [[0.         0.00070802 0.04486435]] 0.8771943898953448


In [20]:
mean_squared_error(y_test,model2.predict(X_test))

189.45602598100356

## Regresión no lineal

In [21]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error


def get_MSE_for_degree_k_model (k):
    pipelined_model = Pipeline([
        ('transform', PolynomialFeatures(degree=k)),
        ('regression',LinearRegression(fit_intercept=True))
    ])
    pipelined_model.fit(X_train[['rooms_revenue']],y_train)
    return mean_squared_error(pipelined_model.predict(X_test[['rooms_revenue']]),y_test)

In [22]:
ks=np.array(range(0,10))
MSEs = [get_MSE_for_degree_k_model(k) for k in ks]
MSEs_and_k =pd.DataFrame({'k':ks,'MSE':MSEs})
MSEs_and_k

,k,MSE
0,0,1451.395583
1,1,191.509976
2,2,148.839867
3,3,143.046668
4,4,246.877123
5,5,430.106203
6,6,656.009146
7,7,921.185601
8,8,1192.812166
9,9,1419.948556


### Se usara el modelo cúbico

In [23]:
X = pd.DataFrame(X_train['rooms_revenue'])

In [24]:
X['x2']= X['rooms_revenue']**2
X['x3']= X['rooms_revenue']**3

In [25]:
X

,rooms_revenue,x2,x3
726335,19285.00,3.719112e+08,7.172308e+12
741345,65886.67,4.341053e+09,2.860175e+14
618107,87094.10,7.585382e+09,6.606420e+14
748550,36560.07,1.336639e+09,4.886761e+13
332351,16175.30,2.616403e+08,4.232111e+12
...,...,...,...
59963,85578.64,7.323704e+09,6.267526e+14
609570,53961.19,2.911810e+09,1.571247e+14
738862,36578.36,1.337976e+09,4.894098e+13
751897,28248.29,7.979659e+08,2.254117e+13


In [26]:
model3=LinearRegression()
model3.fit(X[['rooms_revenue','x2','x3']],y_train)
print(model3.intercept_, model3.coef_, model3.score(X[['rooms_revenue','x2','x3']],y_train))

[3.32023059] [[ 1.16657195e-03 -3.50067205e-09  5.02133199e-15]] 0.903308922231241


In [27]:
X_test = pd.DataFrame(X_test['rooms_revenue'])
X_test['x2']= X_test['rooms_revenue']**2
X_test['x3']= X_test['rooms_revenue']**3

In [28]:
mean_squared_error(y_test,model3.predict(X_test[['rooms_revenue','x2','x3']]))

143.0467127913151

### Análisis ANOVA

In [29]:
from statsmodels.formula.api import ols
model_ols1 = ols('rooms_occupied ~ C(rooms_revenue)', data=datos).fit()

In [37]:
anova_table1 = sm.stats.anova_lm(model_ols1)
anova_table1

,df,sum_sq,mean_sq,F,PR(>F)
C(rooms_revenue),4682.0,7.129029e+06,1522.646086,1081.389465,0.0
Residual,522.0,7.350000e+02,1.408046,NaN,NaN


In [35]:
anova_table = sm.stats.anova_lm(model2)
anova_table

AttributeError: 'LinearRegression' object has no attribute 'model'

In [30]:
model_ols2 = ols('rooms_occupied ~ C(rooms_revenue) + C(compset_rooms_revenue) + C(rooms_revenue):C(compset_rooms_revenue)', data=datos).fit()

In [ ]:
anova_table2 = sm.stats.anova_lm(model_ols2)
anova_table2